Aluno: Gabriel Freitas Santos
MEC 2014

0 - Desenvolvimento do Modelo AR do trabalho anterior

In [1]:
import pandas as pd
import scipy.io as sio
import numpy as np
from os import getcwd,pardir
from os.path import join ,abspath
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook as tqdm
from statsmodels.tsa import ar_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
from scipy import stats
from scipy.stats.stats import mode
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (train_test_split, RepeatedKFold,
RandomizedSearchCV)
from sklearn.linear_model import Ridge
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline


fname = join(abspath(join(getcwd(), pardir))+'\\02_feature','data3SS2009.mat')
mat_contents = sio.loadmat(fname)
dataset = mat_contents['dataset']
N, Chno, Nc = dataset.shape
y = mat_contents['labels'].reshape(Nc)
Ch1 = dataset[:,0,:] # célula de carga: força do shaker
Ch2 = dataset[:,1,:] # acelerômetro: base
Ch3 = dataset[:,2,:] # acelerômetro: 1o andar
Ch4 = dataset[:,3,:] # acelerômetro: 2o andar
Ch5 = dataset[:,4,:] # acelerômetro: 3o andar
FeatAR = []
na=30 #Model Order
for i in tqdm(range(Nc)):
    ARmodel2 = ar_model.AutoReg(Ch2[:,i],na).fit()
    ARmodel3 = ar_model.AutoReg(Ch3[:,i],na).fit()
    ARmodel4 = ar_model.AutoReg(Ch4[:,i],na).fit()
    ARmodel5 = ar_model.AutoReg(Ch5[:,i],na).fit()
    ARparams = np.concatenate([ARmodel2.params,ARmodel3.params,ARmodel4.params,ARmodel5.params])
    FeatAR.append(ARparams)
FeatAR=np.array(FeatAR)
FeatAR.shape # X1
ln,cols = FeatAR.shape
#Montar metodo de normalização
scaler = MinMaxScaler(feature_range=(-1,1))
dfAR = pd.concat([pd.DataFrame(scaler.fit_transform(FeatAR)),pd.DataFrame({'target':y})],axis=1) #Normalização e criação de DF para plot de X1

C:\Program Files\Python38\lib\site-packages\statsmodels\compat\pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
C:\Users\Gabriel Freitas\AppData\Local\Temp\ipykernel_10520\2114589272.py:15: DeprecationWarning: Please use `mode` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import mode


  0%|          | 0/850 [00:00<?, ?it/s]

01 - Design of Hyperparameter search:

Testing 3 different models: softamax, SVM and KNN

In [ ]:
models = []
models.append(('LRG',LogisticRegression(multi_class='multinomial')))
models.append(('SVM',SVC()))
models.append(('KNN',KNeighborsClassifier()))
models

Define parameters to be tested:

In [ ]:
param_grids =[]
param_grids.append( # Softmax
    [
        {
            "LRG__C":[1, 10, 20, 30, 40, 50, 60, 70, 100],
            "LRG__solver" :['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']  
        }
    ]
)
param_grids.append( #SVC
    [
        {
            "C"      : stats.loguniform(1e-1,1e3),
            "kernel" : ['linerar','poly','rbf','sigmoid'],
            "degree" : stats.randint(2,5),
            "gamma"  : stats.loguniform(1e-4,1e0),
            "epsilon"  : stats.loguniform(1e-4,1e1)
        }
    ]
)
param_grids.append( #KNN
    [
        {
            "n_neighbors" : stats.randint(2,100),
            "kernel"      : ["uniform","distance"]            
        }
    ]
)

Get results for default parameters from each model

In [ ]:
kfolds = 5
nkiter = 50
n_iter = 100
acc=[]
cfmx =[]
best_params =[]
for i,md in enumerate(models):
    #Separar Data set em conjuntos de treino e teste ()
    train_x, test__x, train_y, test_y = train_test_split(dfAR[dfAR.columns[0:-1]], dfAR.target, test_size=0.4, random_state=i)

    rkf = RepeatedKFold(n_splits=kfolds,n_repeats=nkiter,random_state=i)

    param_grid = param_grids[i]

    #Cria modelo
    clf = RandomizedSearchCV(md,verbose=0,scoring="accuracy",n_iter=nkiter,n_jobs=-1,cv=rkf,random_state=i,param_distributions=param_grid)
    clf.fit(train_x,train_y)
    best_params[i] = clf.best_params()
    yh_teste_bch = clf.predict(test_y)
    acc[i] = accuracy_score(test_y, yh_teste_bch,normalize=True)
    cfmx[i] = confusion_matrix(test_y, yh_teste_bch)



    